In [ ]:
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import warnings
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False
warnings.filterwarnings(action='ignore')

### 주택소유상태: ANY 삭제
### ID 컬럼 삭제

In [2]:
df = pd.read_csv('C:/Users/wpfke/OneDrive/문서/ML/open/train.csv')

df = df.drop(index=df[df['주택소유상태'] == 'ANY'].index)
df = df.drop(['ID'], axis=1)

In [3]:
def get_period(x):
    if x in ['< 1 year', '<1 year']:
        return 0
    elif x in ['1 year','1 years']:
        return 1
    elif x == '2 years':
        return 2
    elif x == '3 years':
        return 3
    elif x == '4 years':
        return 4
    elif x=='5 years':
        return 5
    elif x=='6 years':
        return 6
    elif x=='7 years':
        return 7
    elif x=='8 years':
        return 8
    elif x=='9 years':
        return 9
    elif x in ['10+ years', '10+years']:
        return 10
    else:
        return 5
df['근로기간'] = df['근로기간'].apply(lambda x: int(get_period(x)))

In [4]:
df['근로기간'].value_counts()

근로기간
10    32480
5     11425
2      8450
0      8144
3      7581
1      6305
4      5588
8      4888
6      3874
7      3814
9      3744
Name: count, dtype: int64

### 표준편차를 사용한 이상치 제거

In [5]:
import numpy as np

def remove_outliers(df, col, threshold=3):
    mean = np.mean(df[col])
    std_dev = np.std(df[col])
    lower_bound = mean - threshold * std_dev
    upper_bound = mean + threshold * std_dev
    df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    
    return df

outlier_columns = ['대출금액', '연간소득', '부채_대비_소득_비율', '총계좌수', '최근_2년간_연체_횟수', '총상환원금', '총상환이자', '총연체금액', '연체계좌수', '근로기간']

for col in outlier_columns:
    df = remove_outliers(df, col)


### 파생변수 생성

In [6]:
df['dv_period'] = df['대출기간'].str.extract('(\d+)').astype(int)
df['dv_ratio'] = (df['총상환원금']) / (df['대출금액'] + 1)
df['dv_short_ability'] = (df['연간소득'] / 12) - (df['대출금액'] / 36)                            
df['dv_left_debt'] = df['대출금액'] - df['총상환원금'] - df['총상환이자']
df['dv_rent_p'] = df['총상환원금'] / (df['대출금액'] + 1)                                        
df['dv_rent_i'] = df['총상환이자'] / (df['대출금액'] + 1)

In [7]:
df = df.drop(columns = ['대출기간', '연간소득'], axis= 1)

In [8]:
scaler = MinMaxScaler()
numeric = df.select_dtypes(exclude= 'object').columns
df[numeric] = scaler.fit_transform(df[numeric])

In [9]:
from sklearn.preprocessing import LabelEncoder

In [10]:
le = LabelEncoder()
object_cols = df.select_dtypes(include = 'object').columns
for col in object_cols:
    if col != '대출등급':
        df[col] = le.fit_transform(df[col])

In [11]:
X = df.drop('대출등급', axis = 1)
y = df['대출등급']

In [12]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import LabelEncoder

In [13]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [14]:
xgb_model = XGBClassifier(random_state=42)
xgb_model.fit(X, y_encoded)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [15]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [16]:
cv_results = cross_val_score(xgb_model, X, y_encoded, cv=kfold, scoring='accuracy')

In [17]:
print("Mean Accuracy: %.2f%%" % (cv_results.mean() * 100))

Mean Accuracy: 94.45%
